In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_excel('clean cses.xlsx')

## Deal with empty cells (fill with NaN)

In [3]:
df=df.replace(r'^\s*$', np.nan, regex=True)

In [64]:
def likeip_vp_df_split(df,country_name_str):
    df=df.replace(r'^\s*$', np.nan, regex=True)
    ## Weight(vp) DataFrame
    Countries=list(df['E1006_NAM'].unique())
    weight_list=list(df.columns[-9:])
    weight_list_countries=['E1006_NAM']+weight_list
    weight_df=df[weight_list_countries].groupby('E1006_NAM').mean(numeric_only=False)
    ## Voters(likeip) DataFrame
    voter_list=list(df.columns[2:-9])
    voter_list_countries=['E1006_NAM']+voter_list
    voter_df=df[voter_list_countries]
    voter_df_ctry=voter_df[voter_df['E1006_NAM']==country_name_str].iloc[:,0:len(vp_cal)+1]
    return voter_df_ctry,weight_df

In [65]:
voter_df,weight_df=likeip_vp_df_split(df,'Brazil')

In [66]:
voter_df

,E1006_NAM,E3017_A,E3017_B,E3017_C,E3017_D,E3017_E,E3017_F,E3017_G,E3017_H,E3017_I
0,Brazil,NaN,1.0,1,NaN,1,NaN,NaN,NaN,NaN
1,Brazil,9.0,0.0,0,NaN,NaN,5,NaN,NaN,NaN
2,Brazil,2.0,0.0,3,NaN,NaN,NaN,2,NaN,NaN
3,Brazil,10.0,0.0,0,NaN,0,0,0,NaN,0
4,Brazil,NaN,0.0,10,5,NaN,5,NaN,NaN,8
...,...,...,...,...,...,...,...,...,...,...
2501,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2502,Brazil,NaN,10.0,10,NaN,NaN,5,3,NaN,5
2503,Brazil,NaN,10.0,5,NaN,10,NaN,5,NaN,5
2504,Brazil,NaN,0.0,3,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
weight_df

,E5001_A,E5001_B,E5001_C,E5001_D,E5001_E,E5001_F,E5001_G,E5001_H,E5001_I
E1006_NAM,,,,,,,,,
Brazil,11.640000,10.29000,5.990000,5.840000,5.570000,5.530000,5.51,5.31,5.07
Chile,17.810000,15.98000,10.280000,9.760000,6.100000,5.720000,4.59,4.26,3.61
Costa Rica,19.490000,18.16000,16.270000,14.600000,7.670000,4.210000,3.95,2.45,2.32
Greece,35.450000,28.08000,6.990000,6.290000,5.550000,4.090000,3.69,3.47,NaN
Hungary,49.270000,19.06000,11.910000,7.060000,5.380000,3.060000,1.73,0.66,NaN
Lithuania,22.630000,22.45000,15.040000,9.450000,6.320000,5.720000,5.55,4.88,NaN
Montenegro,41.410000,20.32000,11.050000,10.010000,5.230000,3.260000,3.16,1.27,0.47
Portugal,38.210000,29.19000,10.010000,6.660000,4.440000,3.490000,1.35,1.35,1.14
Republic of Korea,38.330000,37.00000,14.850000,1.650000,NaN,NaN,NaN,NaN,NaN


## Country spread ##

In [51]:
def country_spread(file_path_str,country_name_str):
    
    ## Load File ##
    file_dir=file_path_str
    df=pd.read_excel(file_dir)
    df=df.replace(r'^\s*$', np.nan, regex=True)
    
    ## Weight(vp) DataFrame
    Countries=list(df['E1006_NAM'].unique())
    weight_list=list(df.columns[-9:])
    weight_list_countries=['E1006_NAM']+weight_list
    weight_df=df[weight_list_countries].groupby('E1006_NAM').mean(numeric_only=False)
    ## Voters(likeip) DataFrame
    voter_list=list(df.columns[2:-9])
    voter_list_countries=['E1006_NAM']+voter_list
    voter_df=df[voter_list_countries]
    
    ## Calculate spread
    
    vp_cal=weight_df.loc[country_name_str].iloc[0:,].dropna().values
    likeip_cal=voter_df[voter_df['E1006_NAM']==country_name_str].iloc[:,1:len(vp_cal)+1].dropna(axis=0)
    like_bar=likeip_cal.dot(vp_cal*0.01)
    likeip_likeBar=(likeip_cal.T-like_bar).T
    square=likeip_likeBar.apply(np.square).round(2)
    sum_vp_dot_squre=square.dot(vp_cal*0.01)
    spread=np.sqrt(np.array(sum_vp_dot_squre,dtype=np.float64))
    likeip_cal['spread']=spread.round(2)
    Contry_spread=pd.DataFrame(likeip_cal['spread'].describe().round(2)).rename(columns={"spread":country_name_str+' spread description'})
    
    ## save data ##
    Contry_spread.to_excel(country_name_str+' Contry_spread.xlsx')
    likeip_cal.to_excel(country_name_str+' likeip_cal.xlsx')
    weight_df.to_excel(country_name_str+' weight_df.xlsx')
    
    return Contry_spread,likeip_cal,vp_cal

In [56]:
Contry_spread,likeip_cal,vp_cal =country_spread('clean cses.xlsx','Chile')

In [57]:
likeip_cal

,E3017_A,E3017_B,E3017_C,E3017_D,E3017_E,E3017_F,E3017_G,E3017_H,E3017_I,spread
2506,0.0,0.0,0,0,0,0,0,0,0,0.00
2508,0.0,0.0,0,6,3,0,10,0,5,2.67
2509,0.0,0.0,0,0,0,5,0,0,0,1.15
2510,6.0,6.0,9,9,9,9,1,5,1,2.51
2511,5.0,6.0,4,5,6,4,7,6,5,1.26
...,...,...,...,...,...,...,...,...,...,...
4498,3.0,4.0,2,1,1,0,0,5,0,1.43
4499,4.0,5.0,6,5,5,6,5,4,4,1.14
4502,1.0,2.0,1,0,0,1,2,0,1,0.67
4504,3.0,3.0,6,10,4,5,5,3,5,2.21


In [58]:
Contry_spread

,Chile spread description
count,1131.00
mean,1.45
std,1.18
min,0.00
25%,0.22
50%,1.37
75%,2.27
max,4.52


In [59]:
vp_cal

array([17.81, 15.98, 10.28,  9.76,  6.1 ,  5.72,  4.59,  4.26,  3.61])